# Laboratório Kafka
## Alunos

**Gabriela da Gama Pivetta - 180052845**

**Murilo Gomes de Souza - 180025601**

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("spark://cm1:7077").appName("testekafka-180025601").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/08 21:49:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/08 21:49:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [82]:
lines = spark.readStream \
        .format("socket") \
        .option("host","cm4") \
        .option("port","9044") \
        .option('includeTimestamp', 'true') \
        .load()

23/05/08 22:15:20 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [83]:
from pyspark.sql.functions import explode, split, col, window, to_json, upper, struct, lit

In [84]:
words = lines \
    .select(
        explode(split(col("value"), "\s+")).alias("word"),
        lines.timestamp
    ).select(
        upper(col("word")).alias("word"),
        col("timestamp")
    )


In [85]:
wordCounts = words.withWatermark("timestamp", '10 seconds') \
    .groupBy(
        window(words.timestamp, '10 seconds', '10 seconds'),
        col("word")
    ).count()

In [97]:
wordCountsJson = wordCounts.select(
        lit('1').alias('key'),
        to_json(struct('word', 'count')).alias('value')
    )

In [95]:
wordCountsJson.writeStream.outputMode('update').format("kafka") \
 .option("kafka.bootstrap.servers", "cm4:9092") \
 .option("topic", "student-a180025601-saida") \
 .option('checkpointLocation', '/tmp/spark-a180025601') \
 .trigger(processingTime='10 seconds') \
 .start()

23/05/08 22:22:57 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
